In [ ]:
%matplotlib inline
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import re
import seaborn as sns

import pandas as pd
import numpy as np
import datetime
import pandas as pd


from sklearn.model_selection import train_test_split

In [ ]:
df_airquality = pd.DataFrame.from_csv('data/airquality.csv')
df_district = pd.DataFrame.from_csv('data/district.csv')
df_meteorology = pd.DataFrame.from_csv('data/meteorology.csv')
df_station = pd.DataFrame.from_csv('data/station.csv')
df_weatherforecast = pd.DataFrame.from_csv('data/weatherforecast.csv')

In [ ]:
final_df = df_airquality.join(df_station)
final_df = final_df.reset_index()
final_df['station_id'] = final_df['\ufeffstation_id']
final_df = final_df.drop('\ufeffstation_id',1)
final_df = final_df.join(df_district, on=['district_id'],rsuffix='_district')

In [ ]:
df_city = pd.DataFrame.from_csv('data/city.csv')
df_city = df_city.reset_index()

df_city['city_id'] = df_city['\ufeffcity_id']
df_city['latitude_city']= df_city['longitude']
df_city['longitude_city']= df_city['longitude']

df_city = df_city.drop('latitude',1)
df_city = df_city.drop('longitude',1)

df_city = df_city.drop('\ufeffcity_id',1)
df_city['city_id'] = df_city.city_id.apply(lambda x : int(x))
final_df = pd.merge(final_df, right=df_city,on=['city_id'],how='left')

In [ ]:
final_df = final_df.drop('name_chinese_x',1)
final_df = final_df.drop('name_chinese_district',1)
final_df = final_df.drop('name_chinese_y',1)
final_df = final_df.drop('name_english_y',1)

In [ ]:
df_meteorology = df_meteorology.reset_index()
df_meteorology['id'] = df_meteorology['\ufeffid']
df_meteorology = df_meteorology.drop('\ufeffid',1)

In [ ]:
df_meteorology['district_id'] = df_meteorology['id']
df_meteorology = df_meteorology.drop('id',1)
final_df = pd.merge(final_df, right=df_meteorology,on=['district_id','time'],how='left')

In [ ]:
final_df['latitude_city']  = final_df['latitude_city'].apply(lambda x : str(x))
final_df['longitude_city']  = final_df['longitude_city'].apply(lambda x : str(x))

final_df['latitude']  = final_df['latitude'].apply(lambda x : str(x))
final_df['longitude']  = final_df['longitude'].apply(lambda x : str(x))

final_df['lat-lon-city'] = final_df['latitude_city'] + ' ' + final_df['longitude_city']
final_df['lat-lon-district'] = final_df['latitude'] + ' ' + final_df['longitude']

In [ ]:
final_df = final_df.drop('latitude_city',1)
final_df = final_df.drop('longitude_city',1)

final_df = final_df.drop('latitude',1)
final_df = final_df.drop('longitude',1)

final_df['time'] = final_df.time.apply(lambda x : datetime.datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S'))

In [ ]:
final_df['year'] = final_df['time'].apply(lambda x : x.year)
final_df['month'] = final_df['time'].apply(lambda x : x.month)
final_df['day'] = final_df['time'].apply(lambda x : x.day)
final_df['hour'] = final_df['time'].apply(lambda x : x.hour)
final_df['minute'] = final_df['time'].apply(lambda x : x.minute)
final_df['seconds'] = final_df['time'].apply(lambda x : x.second)

final_df = final_df.drop('time',1)

In [ ]:
final_df.info()

In [ ]:
# Save it to a temporary data frame 
# final_df.to_csv('data/final_dataframe.csv')

In [ ]:
# final_df = pd.DataFrame.from_csv('data/final_dataframe.csv')

In [ ]:
final_df.columns

In [ ]:
final_df.head()

In [ ]:
train_x_numerical = [ 'year', 'month', 'day', 'hour' , 'minute', 'seconds', 'district_id', 'city_id']
train_x_string = ['name_english_x', 'lat-lon-city', 'lat-lon-district',  'name_english_district', ]           
train_x_weather = ['weather', 'temperature', 'pressure', 'humidity','wind_speed', 'wind_direction']

train_columns = []
train_columns.extend(train_x_numerical)
train_columns.extend(train_x_weather)

train_y_pollution = ['PM25_Concentration', 'PM10_Concentration', 'NO2_Concentration', 'CO_Concentration', 'O3_Concentration', 'SO2_Concentration']


In [ ]:
df_null = final_df[train_columns][final_df[train_columns].isnull().any(axis=1)]
df_null.head(10)

### Trying simple logistic regression for district predictons 

1. Type 1 fill NaN with 0 to see how the things are performing 
2. Remove rows with NaN to see what's the impact 

In [ ]:
df = final_df.fillna(0.0)
train_df, test_df = train_test_split(df, test_size = 0.2)
print(len(train_df) , len(test_df))

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_df[train_columns], train_df[train_y_pollution])

# The coefficients
print('\nCoefficients: \n', regr.coef_)
# The mean squared error
print("\n\nMean squared error for " )
print(np.mean((regr.predict(test_df[train_columns]) - test_df[train_y_pollution]) ** 2))
# Explained variance score: 1 is perfect prediction
print('\nVariance score: %.2f' % regr.score(test_df[train_columns], test_df[train_y_pollution]))

In [ ]:
# Removing entries with NaN values in the regression model
# Helped to impove scores
df = final_df[~final_df.isnull().any(axis=1)]
train_df, test_df = train_test_split(df, test_size = 0.2)
print(len(train_df) , len(test_df))

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_df[train_columns], train_df[train_y_pollution])

# The coefficients
print('\nCoefficients: \n', regr.coef_)
# The mean squared error
print("\n\nMean squared error for " )
print(np.mean((regr.predict(test_df[train_columns]) - test_df[train_y_pollution]) ** 2))
# Explained variance score: 1 is perfect prediction
print('\nVariance score: %.2f' % regr.score(test_df[train_columns], test_df[train_y_pollution]))

#### Using averages to estimate values and rerunning the regressions on the dataframe

In [ ]:
merged_df = final_df.groupby(train_x_numerical).mean().reset_index()
print ("With remove NaN's ", len(final_df[~final_df.isnull().any(axis=1)]))
print ("Without removed Nan's " , len(final_df))
print ("Length of DF with merged ", len(merged_df[~merged_df.isnull().any(axis=1)]))

In [ ]:
# Removing entries with NaN values in the regression model
# Helped to impove scores
df = merged_df[~merged_df.isnull().any(axis=1)].fillna(0.0)
train_df, test_df = train_test_split(df, test_size = 0.2)
print(len(train_df) , len(test_df))

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_df[train_columns], train_df[train_y_pollution])

# The coefficients
print('\nCoefficients: \n', regr.coef_)
# The mean squared error
print("\n\nMean squared error for " )
print(np.mean((regr.predict(test_df[train_columns]) - test_df[train_y_pollution]) ** 2))
# Explained variance score: 1 is perfect prediction
print('\nVariance score: %.2f' % regr.score(test_df[train_columns], test_df[train_y_pollution]))

### Using a more complicated neural network based approach

In [ ]:
from sklearn.neural_network import MLPRegressor 

In [ ]:
df = merged_df[~merged_df.isnull().any(axis=1)].fillna(0.0)
train_df, test_df = train_test_split(df, test_size = 0.2)

print(len(train_df) , len(test_df))

X_train = train_df[train_columns]
Y_train = train_df[train_y_pollution]

X_test = train_df[train_columns]
Y_test = train_df[train_y_pollution]

Multi-layer Perceptron is sensitive to feature scaling, so it is highly recommended to scale your data. For example, scale each attribute on the input vector X to `[0, 1]` or `[-1, +1]`, or standardize it to have mean `0` and variance `1`. Note that you must apply the same scaling to the test set for meaningful results. You can use StandardScaler for standardization.


In [ ]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)  

In [ ]:
print(len(train_df) , len(test_df))
print('Training neural Network model might take some time')
regr.fit( X_train , Y_train)

In [ ]:
print("\n\nMean squared error for " )
print(np.mean((regr.predict(X_test) - Y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('\nVariance score: %.2f' % regr.score(X_test, Y_test))